In [133]:
from __future__ import division
import numpy as np
from scipy import spatial
import pandas as pd
import matplotlib.pyplot as plt
from sko.ACA import ACA_TSP
import six
import sys
sys.modules['sklearn.externals.six'] = six
import mlrose
from typing import DefaultDict
import time

In [73]:
# Greedy
INT_MAX = 2147483647
def findMinRoute(tsp):
    sum = 0
    counter = 0
    j = 0
    i = 0
    min = INT_MAX
    visitedRouteList = DefaultDict(int)
 
    visitedRouteList[0] = 1
    route = [0] * len(tsp)
 
    while i < len(tsp) and j < len(tsp[i]):
 
        if counter >= len(tsp[i]) - 1:
            break
 
        if j != i and (visitedRouteList[j] == 0):
            if tsp[i][j] < min:
                min = tsp[i][j]
                route[counter] = j + 1
 
        j += 1

        if j == len(tsp[i]):
            sum += min
            min = INT_MAX
            visitedRouteList[route[counter] - 1] = 1
            j = 0
            i = route[counter] - 1
            counter += 1
 
    i = route[counter - 1] - 1
 
    for j in range(len(tsp)):
 
        if (i != j) and tsp[i][j] < min:
            min = tsp[i][j]
            route[counter] = j + 1
 
    sum += min
 
    return sum

In [3]:
def cal_total_distance(routine):
    num_points, = routine.shape
    return sum([distance_matrix[routine[i % num_points], routine[(i + 1) % num_points]] for i in range(num_points)])

In [126]:
num_points = 15
points_coordinate = np.random.rand(num_points, 2)
distance_matrix = spatial.distance.cdist(points_coordinate, points_coordinate, metric='euclidean')


In [115]:
points_coordinate

array([[0.73661182, 0.51269964],
       [0.98234008, 0.914696  ],
       [0.83462449, 0.05276104],
       [0.29583003, 0.48146197],
       [0.3967705 , 0.17616197],
       [0.34280323, 0.72158196],
       [0.86826239, 0.31510915],
       [0.05105193, 0.84019243],
       [0.14591896, 0.07445813],
       [0.92643887, 0.47763342]])

In [127]:
aca = ACA_TSP(func=cal_total_distance, n_dim=num_points,
                  size_pop=30, max_iter=30,
                  distance_matrix=distance_matrix)
best_x, best_y = aca.run()

In [119]:
best_x

array([0, 9, 6, 2, 4, 8, 3, 7, 5, 1])

In [128]:
best_y

3.0603134946789967

In [131]:
fitness_coords = mlrose.TravellingSales(coords = points_coordinate)
problem_fit = mlrose.TSPOpt(length = num_points, fitness_fn = fitness_coords,maximize=False)
best_state, best_fitness = mlrose.genetic_alg(problem_fit,pop_size=2000, mutation_prob = 0.2, max_attempts = 30)

print('The best state found is: ', best_state)

print('The fitness at the best state is: ', best_fitness)

The best state found is:  [ 5 13 12  2 11  8  0  3 10  1  4  6 14  7  9]
The fitness at the best state is:  4.0980244192293975


In [132]:
best = findMinRoute(distance_matrix)
print(best)

2.946567815511009


In [142]:
results = {}
res_scatter = {"Algo":[],"N points":[],"Time":[],"Best":[],"N run":[]}
for n in range(5,41,5):
    num_points = n
    points_coordinate = np.random.rand(num_points, 2)
    distance_matrix = spatial.distance.cdist(points_coordinate, points_coordinate, metric='euclidean')
    results["Ant colony "+str(n)+" points BEST"] = []
    results["Ant colony "+str(n)+" points TIME"] = []
    results["Genetic "+str(n)+" points BEST"] = []
    results["Genetic "+str(n)+" points TIME"] = []
    results["Greedy "+str(n)+" points BEST"] = []
    results["Greedy "+str(n)+" points TIME"] = []
    for i in range(0,10):
        aca = ACA_TSP(func=cal_total_distance, n_dim=num_points,size_pop=30, max_iter=30,distance_matrix=distance_matrix)
        startTime_ant_colony = time.time()
        best_x, best_y = aca.run()
        executionTime_ant_colony = (time.time() - startTime_ant_colony)
        results["Ant colony "+str(n)+" points BEST"].append(best_y)
        results["Ant colony "+str(n)+" points TIME"].append(executionTime_ant_colony)
        res_scatter["Algo"].append("Ant colony")
        res_scatter["N points"].append(num_points)
        res_scatter["Time"].append(executionTime_ant_colony)
        res_scatter["Best"].append(best_y)
        res_scatter["N run"].append(i+1)

        fitness_coords = mlrose.TravellingSales(coords = points_coordinate)
        problem_fit = mlrose.TSPOpt(length = num_points, fitness_fn = fitness_coords,maximize=False)
        startTime_genetic = time.time()
        best_state, best_fitness = mlrose.genetic_alg(problem_fit,pop_size=2000, mutation_prob = 0.2, max_attempts = 30)
        executionTime_genetic = (time.time() - startTime_genetic)
        results["Genetic "+str(n)+" points BEST"].append(best_fitness)
        results["Genetic "+str(n)+" points TIME"].append(executionTime_genetic)
        res_scatter["Algo"].append("Genetic")
        res_scatter["N points"].append(num_points)
        res_scatter["Time"].append(executionTime_genetic)
        res_scatter["Best"].append(best_fitness)
        res_scatter["N run"].append(i+1)

        startTime_greedy= time.time()
        best = findMinRoute(distance_matrix)
        executionTime_greedy = (time.time() - startTime_greedy)
        results["Greedy "+str(n)+" points BEST"].append(best)
        results["Greedy "+str(n)+" points TIME"].append(executionTime_greedy)
        res_scatter["Algo"].append("Greedy")
        res_scatter["N points"].append(num_points)
        res_scatter["Time"].append(executionTime_greedy)
        res_scatter["Best"].append(best)
        res_scatter["N run"].append(i+1)

In [145]:
df_results = pd.DataFrame(data=results)
df_results.index.name = "Run"
df_results

,Ant colony 5 points BEST,Ant colony 5 points TIME,Genetic 5 points BEST,Genetic 5 points TIME,Greedy 5 points BEST,Greedy 5 points TIME,Ant colony 10 points BEST,Ant colony 10 points TIME,Genetic 10 points BEST,Genetic 10 points TIME,...,Genetic 35 points BEST,Genetic 35 points TIME,Greedy 35 points BEST,Greedy 35 points TIME,Ant colony 40 points BEST,Ant colony 40 points TIME,Genetic 40 points BEST,Genetic 40 points TIME,Greedy 40 points BEST,Greedy 40 points TIME
Run,,,,,,,,,,,,,,,,,,,,,
0,2.101781,0.185342,2.101781,8.328870,1.723192,0.000042,3.459594,0.374849,3.479298,12.193980,...,11.860759,49.590118,5.554703,0.000794,5.776603,2.227840,14.206049,87.224305,5.993664,0.001179
1,2.101781,0.169584,2.101781,8.442375,1.723192,0.000049,3.459594,0.375289,3.479298,27.014904,...,11.201676,23.743566,5.554703,0.000793,5.650524,2.377327,13.715528,37.554949,5.993664,0.001567
2,2.101781,0.168047,2.101781,8.484900,1.723192,0.000038,3.472207,0.376323,3.514216,12.266992,...,11.672088,72.379997,5.554703,0.000805,5.848114,2.321596,13.781927,49.952655,5.993664,0.001314
3,2.101781,0.198916,2.101781,8.979511,1.723192,0.000038,3.459594,0.378644,3.523454,24.485989,...,12.514719,45.304573,5.554703,0.000802,5.984249,2.346089,14.162082,61.161966,5.993664,0.001183
4,2.101781,0.169684,2.101781,8.010074,1.723192,0.000038,3.459594,0.381763,3.720871,10.544823,...,12.006419,64.647533,5.554703,0.000892,5.868233,2.205813,13.840821,91.807230,5.993664,0.001135
5,2.101781,0.164652,2.101781,7.888015,1.723192,0.000037,3.479298,0.375760,3.511297,21.423244,...,12.620946,40.465747,5.554703,0.000887,5.902530,2.269464,14.141656,35.049655,5.993664,0.001191
6,2.101781,0.166734,2.101781,7.979096,1.723192,0.000042,3.479298,0.374302,3.519393,16.710184,...,11.903311,35.760138,5.554703,0.000991,5.860962,2.253444,14.180833,43.059603,5.993664,0.001216
7,2.101781,0.165801,2.101781,7.959070,1.723192,0.000037,3.459594,0.378006,3.479298,26.344049,...,11.693408,57.708751,5.554703,0.000919,5.885010,3.659844,14.556455,56.248831,5.993664,0.001674
8,2.101781,0.166433,2.101781,8.069458,1.723192,0.000065,3.459594,0.377002,3.459594,25.864868,...,12.212150,80.876195,5.554703,0.001006,5.721808,2.299717,14.617859,32.270499,5.993664,0.001337


In [146]:
df_res_scatter = pd.DataFrame(data=res_scatter)
df_res_scatter

,Algo,N points,Time,Best,N run
0,Ant colony,5,0.185342,2.101781,1
1,Genetic,5,8.328870,2.101781,1
2,Greedy,5,0.000042,1.723192,1
3,Ant colony,5,0.169584,2.101781,2
4,Genetic,5,8.442375,2.101781,2
...,...,...,...,...,...
235,Genetic,40,32.270499,14.617859,9
236,Greedy,40,0.001337,5.993664,9
237,Ant colony,40,2.353009,5.870622,10
238,Genetic,40,45.190032,13.931464,10


In [149]:
df_res_scatter_noise = df_res_scatter.copy()

In [155]:
df_res_scatter_noise["N points"] = df_res_scatter["N points"] + np.random.normal(0, 0.8, size=len(list(df_res_scatter["N points"])))

In [156]:
df_res_scatter_noise

,Algo,N points,Time,Best,N run
0,Ant colony,5.435288,0.185342,2.101781,1
1,Genetic,7.050974,8.328870,2.101781,1
2,Greedy,6.326220,0.000042,1.723192,1
3,Ant colony,5.030252,0.169584,2.101781,2
4,Genetic,5.405098,8.442375,2.101781,2
...,...,...,...,...,...
235,Genetic,38.808766,32.270499,14.617859,9
236,Greedy,38.911194,0.001337,5.993664,9
237,Ant colony,39.092552,2.353009,5.870622,10
238,Genetic,40.214306,45.190032,13.931464,10


In [174]:
df_res_scatter_noise.rename(columns={"Best":"Distance"},inplace=True)

In [183]:
df_res_scatter_noise.rename(columns={"Time":"Run time(s)"},inplace=True)

In [175]:
df_res_scatter_noise["Time"] = df_res_scatter_noise["Time"].apply(lambda x: round(x,4))
df_res_scatter_noise["Distance"] = df_res_scatter_noise["Distance"].apply(lambda x: round(x,4))

In [176]:
df_res_scatter_noise

,Algo,N points,Time,Distance,N run
0,Ant colony,5.435288,0.1853,2.1018,1
1,Genetic,7.050974,8.3289,2.1018,1
2,Greedy,6.326220,0.0000,1.7232,1
3,Ant colony,5.030252,0.1696,2.1018,2
4,Genetic,5.405098,8.4424,2.1018,2
...,...,...,...,...,...
235,Genetic,38.808766,32.2705,14.6179,9
236,Greedy,38.911194,0.0013,5.9937,9
237,Ant colony,39.092552,2.3530,5.8706,10
238,Genetic,40.214306,45.1900,13.9315,10


In [158]:
import plotly.express as px

In [191]:
fig = px.scatter(df_res_scatter_noise, x="N points", y="Distance", color="Algo", symbol="Algo",
                hover_data={'Distance':True,"Algo":True,"Run time(s)":True,"N points":False},height=600, title="Ant colony vs Genetic vs Greedy DISTANCE")
#fig.update_traces(marker_size=7)
fig.write_html('Antcolony_vs_Genetic_vs_Greedy_DISTANCE.html')
fig.show()

In [192]:
fig = px.scatter(df_res_scatter_noise, x="N points", y="Run time(s)", color="Algo", symbol="Algo",title="Ant colony vs Genetic vs Greedy RUN TIME (s)",
                hover_data={'Distance':True,"Algo":True,"Run time(s)":True,"N points":False},height=600)
#fig.update_traces(textposition="bottom right")
fig.write_html('Antcolony_vs_Genetic_vs_Greedy_RUNTIME(s).html')
fig.show()